In [9]:
%pip install torch==2.1.0 transformers==4.30.2 sentence-transformers==2.2.2 huggingface-hub==0.15.1 bertopic==0.17.0 pandas==2.2.0 scikit-learn==1.3.2 keybert==0.8.3

  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39155 sha256=e772854310e885b6293af66292b08b075ad1ab5de68a60849b82f355089e2066
  Stored in directory: c:\users\ilai\appdata\local\pip\cache\wheels\0d\36\2d\e6da0678e185644362647aea93f105241f5d4f63a15d5127b9
Successfully built keybert
  Attempting uninstall: keybert
    Found existing installation: keybert 0.9.0
    Uninstalling keybert-0.9.0:
      Successfully uninstalled keybert-0.9.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, LLMRepresentation
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN



ImportError: cannot import name 'cached_download' from 'huggingface_hub' (c:\Users\Ilai\anaconda3\lib\site-packages\huggingface_hub\__init__.py)

In [ ]:
# === CONFIG ===
CSV_PATH = "..\data\youtube_comments\jack_vs_calley_1000.csv"
COMMENT_COLUMN = "text"
USE_OPENAI = False  # Set to True if you want to test LLM labels (and have API key)

# === Load comments ===
df = pd.read_csv(CSV_PATH)
comments = df[COMMENT_COLUMN].dropna().astype(str).tolist()[:1000]

# === Shared embedding model ===
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# === Clustering baseline ===
def run_topic_model(name, rep_model=None):
    print(f"\n\n===== Running BERTopic with: {name} =====")
    topic_model = BERTopic(
        embedding_model=embedding_model,
        representation_model=rep_model,
        verbose=False
    )
    topics, _ = topic_model.fit_transform(comments)
    top_topics = topic_model.get_topic_info().head(5)
    print(top_topics[["Topic", "Name", "Count"]])

# === Option 1: Default (c-TF-IDF) ===
run_topic_model("c-TF-IDF (default)")

# === Option 2: KeyBERT ===
run_topic_model("KeyBERT", KeyBERTInspired())

# === Option 3: LLM (OpenAI GPT) ===
if USE_OPENAI:
    import openai
    openai.api_key = "your-openai-api-key"

    rep_model_llm = LLMRepresentation(
        model="gpt-3.5-turbo",
        chat=True,
        use_api=True
    )

    run_topic_model("LLM (GPT-3.5)", rep_model_llm)
